## Preprocessing UKB data

### Merge UKB chromosome bed files into a single bed file

In [ ]:
pop="eur"
ukbb_chr_path=$PRS_DATASETS/ukbb/raw/chrs
rm $ukbb_chr_path/mergelist.txt || echo ""
for i in {1..22}; do
    echo ${ukbb_chr_path}/chr${i}_${pop} >> ${ukbb_chr_path}/mergelist.txt
done
plink --merge-list ${ukbb_chr_path}/mergelist.txt --make-bed --out ${ukbb_chr_path}/ds_${pop}.all || ret=$?
echo $ret
if [[ -f ${ukbb_chr_path}/ds_${pop}.all-merge.missnp ]] && [[ ret -ne 0  ]]; then
    echo 'fix bed files'
    for i in {1..22}; do
        plink --bfile ${ukbb_chr_path}/chr${i}_${pop} --exclude ${ukbb_chr_path}/ds_${pop}.all-merge.missnp --make-bed --out ${ukbb_chr_path}/chr${i}_${pop}.fixed || echo "" 
        mv ${ukbb_chr_path}/chr${i}_${pop}.fixed.fam ${ukbb_chr_path}/chr${i}_${pop}.fam
        mv ${ukbb_chr_path}/chr${i}_${pop}.fixed.bim ${ukbb_chr_path}/chr${i}_${pop}.bim
        mv ${ukbb_chr_path}/chr${i}_${pop}.fixed.bed ${ukbb_chr_path}/chr${i}_${pop}.bed
    done

echo "Retry to merge (fixed) chrs files"
plink --exclude ${ukbb_chr_path}/ds_${pop}.all-merge.missnp --merge-list ${ukbb_chr_path}/mergelist.txt --make-bed --out ${ukbb_chr_path}/ds_${pop}.all

fi


### Prepare new pheno files for UKB datasets
- Before running the script below, activate your python virtual environment.

In [ ]:
for pop in "gbr" "sas" "afr"; do
     for pheno in "t2d" "osar" "hfvr" "chol" "ht" "ast"; do
         ( python prepare_pheno_file.py --raw_phenotype_file $PRS_DATASETS/ukbb/ukb_code6.csv --pop_file $PRS_DATASETS/ukbb/pop.panel.${pop} --pheno_file $PRS_DATASETS/ukbb/pheno_${pheno}_${pop} --conditions_file $PRS_DATASETS/ukbb/conditions_${pheno}.json
         echo -e "FID\tIID\tlabel" > $PRS_DATASETS/ukbb/pheno_${pheno}_${pop}.tmp
         tail -n +2 $PRS_DATASETS/ukbb/pheno_${pheno}_${pop} >> $PRS_DATASETS/ukbb/pheno_${pheno}_${pop}.tmp
         mv $PRS_DATASETS/ukbb/pheno_${pheno}_${pop}.tmp $PRS_DATASETS/ukbb/pheno_${pheno}_${pop}
         if [[ -d $PRS_DATASETS/ukbb_${pop} ]]; then 
             cp $PRS_DATASETS/ukbb/pheno_${pheno}_${pop} $PRS_DATASETS/ukbb_${pop}/pheno_${pheno}_ 
         fi 
         echo "done ${pop} ${pheno}!" ) &
     done
done